<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/deepspeed_MII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

legacy
- https://github.com/microsoft/DeepSpeed-MII/tree/main/mii/legacy
- https://towardsdatascience.com/deepspeed-deep-dive-model-implementations-for-inference-mii-b02aa5d5e7f7
- https://www.deepspeed.ai/2022/10/10/mii.html
- [**DeepSpeed Inference: Enabling Efficient Inference of Transformer Models at Unprecedented Scale**](https://arxiv.org/abs/2207.00032)

fastgen
- https://github.com/microsoft/DeepSpeed/blob/master/blogs/deepspeed-fastgen/README.md
- https://github.com/microsoft/DeepSpeed/blob/master/blogs/deepspeed-fastgen/chinese/README.md




In [1]:
!pip install deepspeed-mii

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!ds_report

[2023-12-13 09:10:55,461] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables

In [3]:
!git clone https://github.com/microsoft/DeepSpeed-MII

Cloning into 'DeepSpeed-MII'...
remote: Enumerating objects: 3197, done.
remote: Counting objects: 100% (1806/1806), done.
remote: Compressing objects: 100% (677/677), done.
remote: Total 3197 (delta 1384), reused 1326 (delta 1104), pack-reused 1391
Receiving objects: 100% (3197/3197), 6.35 MiB | 1.30 MiB/s, done.
Resolving deltas: 100% (2079/2079), done.


# DeepSpeed-Legacy

ds mii 以前的组件

https://github.com/microsoft/DeepSpeed-MII/tree/main/mii/legacy





深度学习 (DL) 开源社区在过去几个月中出现了巨大的增长。极其强大的文本生成模型（如 Bloom 176B）或图像生成模型（如稳定扩散）现在可供通过 Hugging Face 等平台访问少量甚至单个 GPU 的任何人使用。尽管开源使人工智能功能的访问变得民主化，但其应用仍然受到两个关键因素的限制：推理延迟和成本。

深度学习模型推理的系统优化已经取得了重大进展，可以大大减少延迟和成本，但这些进展并不容易实现。这种可访问性有限的主要原因是深度学习模型推理环境多种多样，模型的大小、架构、系统性能特征、硬件要求等各不相同。确定适用于给定模型的适当系统优化集并正确应用它们是很重要的。通常超出了大多数数据科学家的能力范围，导致低延迟和低成本推理几乎无法实现。

DeepSpeed-MII 是 DeepSpeed 的一个新的开源 Python 库，旨在使强大模型的低延迟、低成本推理不仅可行，而且易于访问。

- MII 提供对数千种广泛使用的深度学习模型的高度优化实现。
- 与原始实施相比，MII 支持的模型可显着降低延迟和成本。例如，MII 将 Big-Science Bloom 176B 模型的延迟降低了 5.7 倍，同时将成本降低了 40 倍以上。同样，它将部署稳定扩散的延迟和成本降低了 1.9 倍。查看更多详细信息，[了解MII 的详尽延迟和成本分析](https://github.com/microsoft/DeepSpeed-MII/tree/main/mii/legacy#quantifying-latency-and-cost-reduction)。
- 为了实现低延迟/成本推理，MII 利用 DeepSpeed-Inference 的一系列优化，例如transformer的深度融合、用于多 GPU 推理的自动张量切片、使用 ZeroQuant 进行动态量化以及其他几个（请参阅我们的博客文章[了解更多详细信息](https://www.deepspeed.ai/2022/10/10/mii.html)）。
- 凭借最先进的性能，MII 只需几行代码即可通过 AML 在本地和 Azure 上低成本部署这些模型。


![](https://github.com/microsoft/DeepSpeed-MII/blob/main/mii/legacy/docs/images/mii-arch.png?raw=true)
MII 架构，显示 MII 如何使用 DS-Inference 自动优化 OSS 模型，然后使用 GRPC 在本地部署，或使用 AML Inference 在 Microsoft Azure 上部署。

MII 的底层由DeepSpeed-Inference提供支持。根据模型类型、模型大小、批量大小和可用硬件资源，MII 自动应用 DeepSpeed-Inference 中的一组适当的系统优化，以最小化延迟并最大化吞吐量。它通过使用许多预先指定的模型注入策略之一来实现这一点，该策略允许 MII 和 DeepSpeed-Inference 识别底层 PyTorch 模型架构并用优化的实现替换它（见图A）。在此过程中，MII 使其 DeepSpeed-Inference 中的广泛优化自动可用于其支持的数千种流行模型。


MII 可以使用 DeepSpeed-Inference 的两种变体。
- 第一个称为 ds-public，包含此处讨论的大部分 DeepSpeed-Inference 优化，也可以通过我们的开源 DeepSpeed 库获得。
- 第二个称为 ds-azure，提供与 Azure 更紧密的集成，并可通过 MII 向所有 Microsoft Azure 客户提供。我们将运行两个 DeepSpeed-Inference 变体的 MII 分别称为 MII-Public 和 MII-Azure。

虽然这两种变体都比开源 PyTorch 基线提供了显着的延迟和成本降低，但后者为基于生成的工作负载提供了额外的性能优势。此处提供了与 PyTorch 基线以及这两个版本之间的完整延迟和成本优势比较。

云服务部署方式大同小异，aws部署方式差不多

这里介绍第一种方式





## 部署

In [4]:
#MII-Public 可以部署在本地或任何云产品上，只需几行代码。MII 创建一个轻量级 GRPC 服务器来支持这种形式的部署，并为查询提供 GRPC 推理端点。

# 作为示例，这里是Hugging Face 的bigscience/bloom-560m模型的部署：
import mii
mii_configs = {"tensor_parallel": 1, "dtype": "fp16"}
mii.deploy(task="text-generation",
           model="bigscience/bloom-560m",
           deployment_name="bloom560m_deployment",
           mii_config=mii_configs)


[2023-12-13 09:11:19,436] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-12-13 09:12:42,722] [INFO] [deployment.py:75:deploy] ************* MII is using DeepSpeed Optimizations to accelerate your model *************
[2023-12-13 09:12:42,722] [INFO] [deployment.py:75:deploy] ************* MII is using DeepSpeed Optimizations to accelerate your model *************
[2023-12-13 09:12:42,727] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:12:42,728] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter trust_remote_code is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:12:42,735] [INFO] [server.py:38:__init__] Hostfile /job/hostfile no

In [5]:
!lsof -i:50051

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 1480 root   40u  IPv4  62907      0t0  TCP *:50051 (LISTEN)


In [6]:
!ps -ef | grep 1480

root        1480    1423 45 09:12 ?        00:01:21 /usr/bin/python3 -m mii.legacy.launch.multi_gpu_
root        2486     392  0 09:15 ?        00:00:00 /bin/bash -c ps -ef | grep 1480
root        2488    2486  0 09:15 ?        00:00:00 grep 1480


## 查询

In [18]:
# 唯一需要的键是"query"，字典之外的所有其他项目都将作为generatekwargs 传递。对于 Hugging Face 提供的模型，您可以在其生成文档中找到所有可能的参数。
#import nest_asyncio
#nest_asyncio.apply()

import mii
generator = mii.mii_query_handle("bloom560m_deployment")
result = generator.query({"query": ["DeepSpeed is", "Seattle is"]}, do_sample=True, max_new_tokens=30)
print(result)


[2023-12-13 09:24:59,842] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:24:59,843] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter trust_remote_code is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
response: "DeepSpeed is [ ] "
response: "Seattle is The Crown:\n The...\n...\n...\n.........\n...\n...\n...\n...\n...\n...\n......\n...\n......\n...\n......\n...\n...\n...\n...\n......\n"
time_taken: 0.654122353
model_time_taken: -1



## 负载均衡

您可以启动负载平衡器和 MII 服务器的多个副本。当您指定 的值时replica_num，mii.deploy()启动负载均衡器服务器和replica_num副本数。请注意，每个副本均由部署在同一服务器上的tensor_parallel服务器进程组成。
```
mii_configs = {
...
    "tensor_parallel": tensor_parallel,
    "replica_num": replica_num,
    "hostfile": hostfile
}
mii.deploy(...
           mii_config=mii_configs,
           ...)
```

客户端将请求发送到负载均衡器，负载均衡器将请求转发到副本，而不是将请求发送到各个 MII 服务器。目前，负载均衡器实现了简单的循环算法。replica_num当设置为 1 时，负载均衡器充当简单代理。

hostfile是 DeepSpeed 启动器使用的主机文件的路径。当未指定 hostfile 时，DeepSpeed-MII 使用/job/hostfile为 DeepSpeed 定义的默认路径。有关详细信息，请参阅DeepSpeed 的文档。


https://www.deepspeed.ai/getting-started/#resource-configuration-multi-node

## 关闭服务

In [19]:
import mii
mii.terminate("bloom560m_deployment")


[2023-12-13 09:25:31,531] [INFO] [terminate.py:12:terminate] Terminating server for bloom560m_deployment
[2023-12-13 09:25:31,531] [INFO] [terminate.py:12:terminate] Terminating server for bloom560m_deployment
[2023-12-13 09:25:31,535] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:25:31,536] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter trust_remote_code is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:25:31,550] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:25:31,551] [WARNING] [confi

## chat

https://github.com/microsoft/DeepSpeed-MII/blob/main/mii/legacy/examples/local/chat/README.md


**训练笔记：https://github.com/weedge/doraemon-nb/blob/main/ds_examples_chatbot.ipynb**


使用[DeepSpeed-Chat](https://github.com/microsoft/DeepSpeedExamples/blob/master/applications/DeepSpeed-Chat/README.md)通过RLHF训练的模型，已经上传至huggingface:

https://huggingface.co/AdamG012/chat-opt-1.3b-rlhf-actor-deepspeed

In [50]:
# deploy start server
!cd DeepSpeed-MII/mii/legacy/examples/local/chat && (nohup python chat-server-example.py &)


nohup: appending output to 'nohup.out'


In [56]:
!cat DeepSpeed-MII/mii/legacy/examples/local/chat/nohup.out

[2023-12-13 09:45:03,835] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-12-13 09:45:05.726142: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 09:45:05.726202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 09:45:05.726229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 09:45:06.874407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Deploying AdamG012/chat-opt-1.3b-rlhf-actor-deepspeed...
[2023-12-13 09:45:44,816] [INFO] [deployment.py:75:deploy] **

In [57]:
!lsof -i:50051

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 10919 root   40u  IPv4 262091      0t0  TCP *:50051 (LISTEN)


In [59]:
# cliet chat
!cd DeepSpeed-MII/mii/legacy/examples/local/chat && python chat-client-example.py


[2023-12-13 09:59:01,271] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-12-13 09:59:03.188707: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 09:59:03.188760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 09:59:03.188785: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 09:59:04.323178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-12-13 09:59:05,180] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is d

In [60]:
import mii
mii.terminate("chat_example_deployment")


[2023-12-13 10:01:33,497] [INFO] [terminate.py:12:terminate] Terminating server for chat_example_deployment
[2023-12-13 10:01:33,497] [INFO] [terminate.py:12:terminate] Terminating server for chat_example_deployment
[2023-12-13 10:01:33,501] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 10:01:33,502] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter trust_remote_code is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 10:01:33,540] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 10:01:33,541] [WARNING] 

In [61]:
!lsof -i:50051

## more
更多示例：
https://github.com/microsoft/DeepSpeed-MII/tree/main/mii/legacy/examples/local

In [21]:
!ls -lh DeepSpeed-MII/mii/legacy/examples/local

total 64K
drwxr-xr-x 2 root root 4.0K Dec 13 09:11 chat
-rw-r--r-- 1 root root  302 Dec 13 09:11 conversational-example.py
-rw-r--r-- 1 root root  757 Dec 13 09:11 conversational-query-example.py
-rw-r--r-- 1 root root  677 Dec 13 09:11 fill-mask-example.py
-rw-r--r-- 1 root root  351 Dec 13 09:11 question-answering-example.py
-rw-r--r-- 1 root root  386 Dec 13 09:11 question-answering-query-example.py
-rw-r--r-- 1 root root  317 Dec 13 09:11 text-classification-example.py
-rw-r--r-- 1 root root  410 Dec 13 09:11 text-classification-query-example.py
-rw-r--r-- 1 root root  324 Dec 13 09:11 text-generation-bloom560m-example.py
-rw-r--r-- 1 root root  456 Dec 13 09:11 text-generation-bloom-example.py
-rw-r--r-- 1 root root  721 Dec 13 09:11 text-generation-fbopt-example.py
-rw-r--r-- 1 root root  568 Dec 13 09:11 text-generation-query-example.py
-rw-r--r-- 1 root root 1.4K Dec 13 09:11 text-generation-zero-example.py
-rw-r--r-- 1 root root  288 Dec 13 09:11 token-classification-example.p

In [25]:
!cd DeepSpeed-MII/mii/legacy/examples/local && (nohup python conversational-example.py &)


nohup: appending output to 'nohup.out'


In [42]:
!cat DeepSpeed-MII/mii/legacy/examples/local/nohup.out

[2023-12-13 09:34:21,814] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-12-13 09:34:23.741561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 09:34:23.741615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 09:34:23.741646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 09:34:24.887416: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Deploying microsoft/DialoGPT-large...
[2023-12-13 09:35:02,358] [INFO] [deployment.py:75:deploy] ************* MII is 

In [47]:
!cd DeepSpeed-MII/mii/legacy/examples/local && python conversational-query-example.py

[2023-12-13 09:41:57,043] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-12-13 09:41:58.932739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 09:41:58.932786: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 09:41:58.932813: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 09:42:00.069422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Querying microsoft/DialoGPT-large...
[2023-12-13 09:42:00,919] [WARNING] [config_utils.py:69:_process_deprecated_field

In [45]:
!lsof -i:50051

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 7710 root   23u  IPv4 190929      0t0  TCP localhost:45268->localhost:50051 (ESTABLISHED)
python3 7844 root   39u  IPv4 185094      0t0  TCP *:50051 (LISTEN)
python3 7844 root   40u  IPv4 189060      0t0  TCP localhost:50051->localhost:45268 (ESTABLISHED)


In [48]:
import mii
mii.terminate("microsoft/DialoGPT-large_deployment")


[2023-12-13 09:43:59,850] [INFO] [terminate.py:12:terminate] Terminating server for microsoft/DialoGPT-large_deployment
[2023-12-13 09:43:59,850] [INFO] [terminate.py:12:terminate] Terminating server for microsoft/DialoGPT-large_deployment
[2023-12-13 09:43:59,854] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:43:59,855] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter trust_remote_code is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 09:43:59,861] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter hf_auth_token is deprecated. Parameter will be removed. Please use the `pipeline_kwargs` field to pass kwargs to the HuggingFace pipeline creation.
[2023-12-13 

In [49]:
!lsof -i:50051

# DeepSpeed-FastGen

https://github.com/microsoft/DeepSpeed/blob/master/blogs/deepspeed-fastgen/chinese/README.md

ds-mii-fastgen 是为了解决LLMs推理加速，提高推理吞吐，以及长prompt问题(llm已经支持长token,树万token)。

GPT-4 和 LLaMA 这样的大型语言模型（LLMs）已在各个层次上成为了集成 AI 的主流服务应用。从常规聊天模型到文档摘要，从自动驾驶到各个软件中的Copilot功能，这些模型的部署和服务需求正在迅速增加。像 DeepSpeed、PyTorch 和其他几个框架可以在 LLM 训练期间实现良好的硬件利用率。但它们在与用户互动及处理开放式文本生成等任务时，受限于这些操作的计算密集度相对较低，现有系统往往在推理吞吐量上遇到瓶颈。

为了解决这一问题， vLLM 这样由 PagedAttention 驱动的框架和 [Orca](https://www.usenix.org/system/files/osdi22-yu.pdf) 这样的系统显著提高了 LLM 推理的性能。然而，这些系统在面对长提示的工作负载时，依旧难以提供良好的服务质量。随着越来越多的模型（例如 MPT-StoryWriter）和系统（例如DeepSpeed Ulysses）支持延伸到数万个令牌的上下文窗口，这些长提示工作负载变得越来越重要。为了更好地理解问题，我们在下文中提供了详细的示例来说明 LLM 的文本生成是如何在“提示处理”和“生成”的这两个阶段中工作的。当系统将它们视为不同的阶段时，生成阶段将被提示处理所抢占，这可能会破坏服务级别协议（SLAs）。

tips:
- vllm 跟进 https://github.com/vllm-project/vllm/issues/1562

DeepSpeed-FastGen 框架，它通过采用动态 SplitFuse 技术，能够提供比vLLM 等先进系统高出多达 2.3 倍的有效吞吐量。DeepSpeed-FastGen 是 DeepSpeed-MII 和 DeepSpeed-Inference 的结合，提供了一个易于使用的服务系统



DeepSpeed-FastGen 是 [DeepSpeed-MII](https://github.com/microsoft/DeepSpeed-MII) 和 [DeepSpeed-Inference](https://github.com/microsoft/DeepSpeed) 的协同组合，如下图所示。这两个软件包共同提供了系统的各个组成部分，包括前端 API、用于使用动态 SplitFuse 调度批次的主机和设备基础设施、优化的内核实现，以及构建新模型实现的工具。
![](https://github.com/microsoft/DeepSpeed/blob/master/blogs/deepspeed-fastgen/assets/images/fastgen-arch-light.png?raw=true)

# 部署选项
示例均可在 DeepSpeedExamples 中运行。安装后，有两种部署方式：交互式非持久管道或持久化服务部署：

## 非持久管道
非持久管道部署是快速入门的好方法，只需几行代码即可完成。非持久模型只在您运行的 python 脚本期间存在，适用于临时交互式会话。



In [ ]:
# https://github.com/microsoft/DeepSpeed-MII/issues/273 need SM>=8.0 (Ampere+) A100
# if use T4 GPU , need use https://github.com/microsoft/DeepSpeed-MII/tree/main/mii/legacy
from mii import pipeline
pipe = pipeline("mistralai/Mistral-7B-v0.1")
output = pipe(["Hello, my name is", "DeepSpeed is"], max_new_tokens=128)
print(output)


## 持久部署
持久部署非常适合用于长时间运行和生产的应用。持久部署使用了轻量级的 GRPC 服务器，可以使用以下两行代码创建：



In [ ]:
import mii
mii.serve("mistralai/Mistral-7B-v0.1")


In [ ]:
# 上述服务器可以同时被多个客户端查询，这要归功于 DeepSpeed-MII 内置的负载平衡器。创建客户端也只需要两行代码：

client = mii.client("mistralai/Mistral-7B-v0.1")
output = client.generate("Deepspeed is", max_new_tokens=128)
print(output)


In [ ]:
#持久部署可以在不再需要时终止：

client.terminate_server()
